# PKGS

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.float_format', '{:,.4f}'.format)

# Creación por chunks

In [2]:
data = pd.read_excel('PruebaDatosVaRAjustado.xlsx', sheet_name=1).set_index('Date').sort_index()
data.tail()

,Bid,Ask,Bid.1,Ask.1
Date,,,,
2025-02-25,110.5156,110.5312,117.6250,117.6562
2025-02-26,110.7656,110.7969,118.1875,118.2188
2025-02-27,110.8125,110.8281,117.7812,117.8125
2025-02-28,111.1719,111.1875,118.3438,118.4062
2025-03-03,111.6406,111.6562,119.1875,119.2188


In [3]:
assets = ['TB10Y', 'TB30Y']

data.columns = [f"{col}_{asset}" for asset, (col1, col2) in zip(assets, zip(data.columns[::2], data.columns[1::2])) for col in (col1, col2)]
data.columns = data.columns.str.replace(r"(Bid|Ask).*?_","\\1_", regex=True)
data.tail()

,Bid_TB10Y,Ask_TB10Y,Bid_TB30Y,Ask_TB30Y
Date,,,,
2025-02-25,110.5156,110.5312,117.6250,117.6562
2025-02-26,110.7656,110.7969,118.1875,118.2188
2025-02-27,110.8125,110.8281,117.7812,117.8125
2025-02-28,111.1719,111.1875,118.3438,118.4062
2025-03-03,111.6406,111.6562,119.1875,119.2188


In [4]:
for i in range(len(assets)):
    data[f"Mid_{assets[i]}"] = (data[f"Ask_{assets[i]}"] + data[f"Bid_{assets[i]}"]) / 2
    data[f"Spread_{assets[i]}"] = (data[f"Ask_{assets[i]}"] - data[f"Bid_{assets[i]}"]) / data[f"Mid_{assets[i]}"]
    data[f"Rend_{assets[i]}"] = data[f"Mid_{assets[i]}"].pct_change().dropna()

data.tail()

,Bid_TB10Y,Ask_TB10Y,Bid_TB30Y,Ask_TB30Y,Mid_TB10Y,Spread_TB10Y,Rend_TB10Y,Mid_TB30Y,Spread_TB30Y,Rend_TB30Y
Date,,,,,,,,,,
2025-02-25,110.5156,110.5312,117.6250,117.6562,110.5234,0.0001,0.0064,117.6406,0.0003,0.0121
2025-02-26,110.7656,110.7969,118.1875,118.2188,110.7812,0.0003,0.0023,118.2031,0.0003,0.0048
2025-02-27,110.8125,110.8281,117.7812,117.8125,110.8203,0.0001,0.0004,117.7969,0.0003,-0.0034
2025-02-28,111.1719,111.1875,118.3438,118.4062,111.1797,0.0001,0.0032,118.3750,0.0005,0.0049
2025-03-03,111.6406,111.6562,119.1875,119.2188,111.6484,0.0001,0.0042,119.2031,0.0003,0.0070


In [5]:
titulos = [100000000, 90000000]

posiciones = [data[f"Mid_{assets[i]}"].iloc[-1] * titulos[i] for i in range(len(assets))]

weights = posiciones / sum(posiciones)

rend_port = data[[f"Rend_{asset}" for asset in assets]].dot(weights).dropna()

long = False
conf = 99

var = np.percentile(rend_port.values,100-conf) if long else np.percentile(rend_port.values,conf)
cvar = rend_port[rend_port < var].mean() if long else rend_port[rend_port > var].mean()

#pd.set_option('display.float_format', '{:,.4f}'.format)

var_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var * sum(posiciones), cvar * sum(posiciones)]})
var_df

,Métrica,Porcentual (%),Monetario ($)
0,VaR,1.6074,"351,914,316.5583"
1,CVaR,1.8068,"395,554,861.5311"


In [6]:
CL_estr = [np.percentile([data[f"Spread_{asset}"]], 99) for asset in assets]
CL_estr_m = [CL_estr[i] * posiciones[i] for i in range(len(assets))]
CL_estr_df = pd.DataFrame({"CL_estr": assets, "Porcentual (%)": np.array(CL_estr)*100, "Monetario ($)": CL_estr_m})
CL_estr_df

,CL_estr,Porcentual (%),Monetario ($)
0,TB10Y,0.0290,"3,240,128.4630"
1,TB30Y,0.0793,"8,511,584.1758"


In [7]:
var_apl_estr = pd.DataFrame({"ApL Estresado": ["VaR", "CVaR"], "Porcentual (%)": [(var + np.dot(CL_estr, weights))*100, (cvar + np.dot(CL_estr, weights))*100], "Monetario ($)": [var * sum(posiciones) + sum(CL_estr_m), cvar * sum(posiciones) + sum(CL_estr_m)]})
var_apl_estr

,ApL Estresado,Porcentual (%),Monetario ($)
0,VaR,1.6611,"363,666,029.1972"
1,CVaR,1.8604,"407,306,574.1699"


In [8]:
CL_prom = [np.mean(data[f"Spread_{asset}"]) for asset in assets]
CL_prom_m = [CL_prom[i] * posiciones[i] for i in range(len(assets))]
CL_prom_df = pd.DataFrame({"CL_prom": assets, "Porcentual (%)": np.array(CL_prom)*100, "Monetario ($)": CL_prom_m})
CL_prom_df

,CL_prom,Porcentual (%),Monetario ($)
0,TB10Y,0.0164,"1,826,601.8639"
1,TB30Y,0.0335,"3,590,112.5196"


In [9]:
var_apl_prom = pd.DataFrame({"ApL Promedio": ["VaR", "CVaR"], "Porcentual (%)": [(var + np.dot(CL_prom, weights))*100, (cvar + np.dot(CL_prom, weights))*100], "Monetario ($)": [var * sum(posiciones) + sum(CL_prom_m), cvar * sum(posiciones) + sum(CL_prom_m)]})
var_apl_prom

,ApL Promedio,Porcentual (%),Monetario ($)
0,VaR,1.6322,"357,331,030.9419"
1,CVaR,1.8315,"400,971,575.9146"


# Función

In [10]:
def VaR_ApL(data, assets, titulos, long, conf):
     data.columns = [f"{col}_{asset}" for asset, (col1, col2) in zip(assets, zip(data.columns[::2], data.columns[1::2])) for col in (col1, col2)]
     data.columns = data.columns.str.replace(r"(Bid|Ask).*?_","\\1_", regex=True)

     for i in range(len(assets)):
          data[f"Mid_{assets[i]}"] = (data[f"Ask_{assets[i]}"] + data[f"Bid_{assets[i]}"]) / 2
          data[f"Spread_{assets[i]}"] = (data[f"Ask_{assets[i]}"] - data[f"Bid_{assets[i]}"]) / data[f"Mid_{assets[i]}"]
          data[f"Rend_{assets[i]}"] = data[f"Mid_{assets[i]}"].pct_change().dropna()
     
     posiciones = [data[f"Mid_{assets[i]}"].iloc[-1] * titulos[i] for i in range(len(assets))]
     weights = posiciones / sum(posiciones)
     rend_port = data[[f"Rend_{asset}" for asset in assets]].dot(weights).dropna()

     var = np.percentile(rend_port.values,100-conf) if long else np.percentile(rend_port.values,conf)
     cvar = rend_port[rend_port < var].mean() if long else rend_port[rend_port > var].mean()
     var_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], 
                            "Porcentual (%)": [var * 100, cvar * 100], 
                            "Monetario ($)": [var * sum(posiciones), cvar * sum(posiciones)]})

     CL_prom = [np.mean(data[f"Spread_{asset}"]) for asset in assets]
     CL_prom_m = [CL_prom[i] * posiciones[i] for i in range(len(assets))]
     apl_prom_p = [(var - np.dot(CL_prom, weights))*100, (cvar - np.dot(CL_prom, weights))*100] if long else [(var + np.dot(CL_prom, weights))*100, (cvar + np.dot(CL_prom, weights))*100]
     apl_prom_m = [var * sum(posiciones) - sum(CL_prom_m), cvar * sum(posiciones) - sum(CL_prom_m)] if long else [var * sum(posiciones) + sum(CL_prom_m), cvar * sum(posiciones) + sum(CL_prom_m)]
     var_apl_prom = pd.DataFrame({"ApL Promedio": ["VaR", "CVaR"], 
                                  "Porcentual (%)": apl_prom_p, 
                                  "Monetario ($)": apl_prom_m
     })

     CL_estr = [np.percentile([data[f"Spread_{asset}"]], 99) for asset in assets]
     CL_estr_m = [CL_estr[i] * posiciones[i] for i in range(len(assets))]
     apl_estr_p = [(var - np.dot(CL_estr, weights))*100, (cvar - np.dot(CL_estr, weights))*100] if long else [(var + np.dot(CL_estr, weights))*100, (cvar + np.dot(CL_estr, weights))*100]
     apl_estr_m = [var * sum(posiciones) - sum(CL_estr_m), cvar * sum(posiciones) - sum(CL_estr_m)] if long else [var * sum(posiciones) + sum(CL_estr_m), cvar * sum(posiciones) + sum(CL_estr_m)]
     var_apl_estr = pd.DataFrame({"ApL Estresado": ["VaR", "CVaR"], 
                                  "Porcentual (%)": apl_estr_p, 
                                  "Monetario ($)": apl_estr_m
     })
     
     return display(var_df), display(var_apl_prom), display(var_apl_estr)


# Prueba clase

In [11]:
data = pd.read_excel('PruebaDatosVaRAjustado2.xlsx', sheet_name=0).set_index('Date').sort_index()
assets = ['Crudo', 'GasNatural']
titulos = [1245*1000, 871*10000]
long = True
conf = 99

VaR_ApL(data, assets, titulos, long, conf)

,Métrica,Porcentual (%),Monetario ($)
0,VaR,-5.9643,"-7,268,538.2184"
1,CVaR,-7.0678,"-8,613,290.5188"


,ApL Promedio,Porcentual (%),Monetario ($)
0,VaR,-6.3472,"-7,735,139.8749"
1,CVaR,-7.4507,"-9,079,892.1753"


,ApL Estresado,Porcentual (%),Monetario ($)
0,VaR,-8.3526,"-10,179,127.4718"
1,CVaR,-9.4561,"-11,523,879.7722"


(None, None, None)

# Prueba tarea

## FX

In [12]:
data = pd.read_excel('DataTarea5.xlsx', sheet_name=0).set_index('Date').sort_index()
assets = ['USD/MXN', 'JPY/MXN', 'EUR/MXN']
titulos = [3200000, 10000000, 5300000]
long = False
conf = 99

VaR_ApL(data, assets, titulos, long, conf)

,Métrica,Porcentual (%),Monetario ($)
0,VaR,2.6134,"4,656,957.0170"
1,CVaR,3.6356,"6,478,384.8304"


,ApL Promedio,Porcentual (%),Monetario ($)
0,VaR,2.6569,"4,734,440.2541"
1,CVaR,3.6790,"6,555,868.0675"


,ApL Estresado,Porcentual (%),Monetario ($)
0,VaR,2.7076,"4,824,739.6518"
1,CVaR,3.7297,"6,646,167.4652"


(None, None, None)

## Derivado

In [13]:
data = pd.read_excel('DataTarea5.xlsx', sheet_name=1).set_index('Date').sort_index()
assets = ['Derivado0']
titulos = [100*37500]
long = True
conf = 99

VaR_ApL(data, assets, titulos, long, conf)

,Métrica,Porcentual (%),Monetario ($)
0,VaR,-4.9077,"-588,274.1440"
1,CVaR,-6.0962,"-730,740.9163"


,ApL Promedio,Porcentual (%),Monetario ($)
0,VaR,-4.9759,"-596,455.2298"
1,CVaR,-6.1644,"-738,922.0021"


,ApL Estresado,Porcentual (%),Monetario ($)
0,VaR,-5.1081,"-612,305.6673"
1,CVaR,-6.2967,"-754,772.4396"


(None, None, None)